### Imports

In [38]:
from dataclasses import dataclass
from pathlib import Path
from Phishing_Detector.utils import *
from Phishing_Detector.constants import *
from urllib import request
from scipy.io import arff
import pandas as pd

In [40]:
#os.chdir('../')
os.chdir('../DS_Projects/Phishing-Domain-Detection/')
os.getcwd()

'c:\\DS_Projects\\Phishing-Domain-Detection'

### Config Entity

In [47]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_dir_path : Path
    processed_data_dir_path : Path
    local_data_file_path : Path
    processed_data_file_path : Path


### Configuration

In [59]:
class ConfigurationManager:

    def __init__(
            self, 
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_dir_path= config.local_data_dir_path,
            processed_data_dir_path= config.processed_data_dir_path,
            local_data_file_path= config.local_data_file_path,
            processed_data_file_path= config.processed_data_file_path
        )

        return data_ingestion_config



In [60]:
class DataIngestion:
    
    def __init__(self, config:DataIngestionConfig):
        self.config = config
        create_directories([config.local_data_dir_path])
        create_directories([config.processed_data_dir_path])

    def download_file(self):
        if not os.path.exists(self.config.local_data_file_path):
            request.urlretrieve(
                url = self.config.source_url,
                filename= self.config.local_data_file_path
            )
        else:
            pass

    def convert_to_csv(self):

        if not os.path.exists(self.config.processed_data_file_path):
            data = arff.loadarff(Path(self.config.local_data_file_path))
            data= pd.DataFrame(data[0])
            data = data.apply(lambda x: x.str.decode('utf8'))
            data = data.apply(pd.to_numeric, errors = 'ignore')
            data.to_csv(path_or_buf= self.config.processed_data_file_path, index=False)
        else:
            pass



In [61]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.convert_to_csv()
except Exception as e:
    raise e